In [ ]:
import numpy as np
import os
import librosa
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
from scipy import signal
from scipy import fft
import librosa.display
import IPython.display as ipd

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
audio_keparoicamL_path = os.path.join(DATA_FOLDER, "keparoicam_clipL.wav")
audio_keparoicamR_path = os.path.join(DATA_FOLDER, "keparoicam_clipR.wav")
whistle_path = os.path.join(DATA_FOLDER, "whistle.wav")

In [ ]:
target_left = audio_keparoicamL_path
target_right = audio_keparoicamR_path

In [ ]:
def plot_audio(data, samplerate=44100):
    plt.figure(figsize=(14, 5))
    librosa.display.waveshow(data, sr=samplerate)
    #plt.title("Audio")
    #plt.plot(data)
    plt.show()

In [ ]:
def stereo_to_mono(wav_array: np.ndarray):
    mono_wav = wav_array.mean(axis=1)
    return mono_wav


def normalize_audio(wav_array: np.ndarray, bits=16.):
    max_value = 2**bits
    normalized_audio = (wav_array/max_value)*2
    return normalized_audio

def butter_lowpass_filter(data: np.ndarray, cutoff: float, samplerate: float, order: int = 5) -> np.ndarray:
    nyq = 0.5 * samplerate
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)
    return y


def butter_highpass_filter(data: np.ndarray, cutoff: float, samplerate: float, order: int = 5) -> np.ndarray:
    nyq = 0.5 * samplerate
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
def filter_whistles(audio_file_path: str):
    samplerate, audio = read(audio_file_path)
    audio_data = butter_lowpass_filter(audio_data, 5000, samplerate)
    audio_data = butter_highpass_filter(audio_data, 1000, samplerate)

    return audio_data

In [ ]:
def fft_conversion(data: np.ndarray, samplerate: float = 44100):
    normalized_audio = normalize_audio(data)
    fft_audio = fft.fft(normalized_audio) 
    return fft_audio

In [ ]:
samplerate, audio = read(whistle_path)
length = audio.shape[0] / samplerate
time = np.linspace(0., length, audio.shape[0])
audio = stereo_to_mono(audio)

In [ ]:
plot_audio(audio, samplerate)

In [ ]:
ipd.Audio(audio, rate=44100)

In [ ]:
audio_fft = fft_conversion(audio)

In [ ]:
k = np.arange(len(audio_fft))
T = len(audio_fft)/samplerate
frequency_label = k/T

In [ ]:
plt.figure(figsize=(16, 16))
plt.plot(frequency_label, abs(audio_fft),'r')
plt.show()

In [ ]:
xf = fft.fftfreq(audio_fft.size, 1/samplerate) 

# show transformed signal (frequencies domain)
plt.figure(figsize=(32, 16))
plt.plot(xf, abs(audio_fft)/np.linalg.norm(audio_fft))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.xticks(np.arange(min(xf), max(xf)+1, 1000.0))
plt.title("Frequency domain signal")
plt.show()

In [ ]:
np.arange(min(xf), max(xf)+1, 3000.0)

In [ ]:
threshold = 0.5
filtered = np.copy(audio_fft)
filtered[abs(audio_fft) < threshold] = 0

# show filtered transformed signal
plt.plot(xf,abs(filtered)/np.linalg.norm(filtered))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.title("FILTERED time domain signal")
plt.show()

In [ ]:
plt.figure(figsize=(16, 16))
plt.plot(frequency_label, abs(filtered)/np.linalg.norm(filtered),'r')
plt.show()

In [ ]:
filtered = fft.ifft(filtered)
    
# show original signal filtered
plt.plot(time, filtered)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Filtered signal")
plt.show()

In [ ]:
ipd.Audio(filtered, rate=44100)